In [1]:
import neuraltda.test_pipeline as ttop
import numpy as np
import neuraltda.topology as top
import h5py as h5
reload(ttop)
reload(top)

/Users/brad/anaconda/envs/neuraltda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


<module 'neuraltda.topology' from '/Users/brad/GentnerLab/code/NeuralTDA/neuraltda/topology.pyc'>

In [ ]:
block_path = '/Users/brad/experiments/B999/'
kwikfile = 'B999_S00_P00.kwik'
bin_id = '0801161542'
bin_def_file = '/Users/brad/GentnerLab/code/NeuralTDA/standard_good_binnings.txt'
n_cells = 60
maxt = 6
fs = 24000.0
dthetadt = 2*np.pi/6
kappa = 1
maxfr = 25
ncellperm = 40
nperms = 1


In [ ]:
spikes, clusters, trials = ttop.generate_test_dataset(n_cells, maxt, fs, dthetadt, kappa, maxfr)

In [ ]:
spikes

In [ ]:
popvec_fname = '/Users/brad/experiments/B999/testbin.binned'
segment_info = {'period': 1}
cluster_group = ['Good']
win_size = 50.0
top.build_population_embedding(spikes, trials, clusters, win_size, fs, cluster_group, segment_info, popvec_fname)

In [ ]:
ttop.test_pipeline(block_path, kwikfile, bin_id, bin_def_file, n_cells, maxt, fs, dthetadt, kappa, maxfr, ncellperm, nperms)

In [ ]:
import pickle


In [ ]:
pickle.load('/Volumes/gentner/btheilma/experiments/B999/topology/0801161509/B999_S00_P00-good-50ms-stim-permuted/B999_S00_P00-good-50ms-stim-permuted-0801161509-stim-test_pipeline_stimulus-bettiPersistence.pkl')

In [2]:
binned_file = '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site03_Z3000__B1235_cat_P02_S03_1/binned_data/1608221332/B1235_cat_P02_S03_1-good-50ms-stim.binned'

In [3]:
bdf = h5.File(binned_file, 'r')

In [4]:
bds = bdf['A_scaled_burung']['0']
fs = bdf.attrs['fs']
nclus = bdf.attrs['nclus']

In [ ]:
np.round(6.5/0.049)

In [ ]:
C = top.calc_corr_raw(bds['pop_vec'], bds['windows'], fs, 1, 4, 1)

In [ ]:
C

In [6]:
wins = bds['windows']

In [ ]:
wins

In [ ]:
wins[0, 1]

In [ ]:
wins[0, 0]

In [ ]:
wins[0,1]-wins[0,0]

In [ ]:
fs

In [7]:
cij = top.compute_Cij_matrix(bds['pop_vec'], wins, fs, nclus, 1)

/Users/brad/GentnerLab/code/NeuralTDA/neuraltda/topology.py:1566: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  cij = np.sum(corr_ij[0:int_lim])
/Users/brad/GentnerLab/code/NeuralTDA/neuraltda/topology.py:1567: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  cji = np.sum(corr_ji[0:int_lim])


In [37]:
cij_r = cij+np.diag(47*[np.nan])
cij_t = cij_r[~(np.isnan(cij_r).all(axis=1))]

In [38]:
cij_tt = cij_t[~(np.isnan(cij_t).all(axis=0))]

/Users/brad/anaconda/envs/neuraltda/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 43 but corresponding boolean dimension is 47
  if __name__ == '__main__':


IndexError: index 43 is out of bounds for axis 0 with size 43

In [51]:
cols_to_remove = np.isnan(cij[0, :])

In [55]:
cij_t = cij[~cols_to_remove]

In [58]:
cij_tt = cij_t[:, ~cols_to_remove]

In [33]:
0+np.nan

nan

In [35]:
cij+np.diag(47*[np.nan])

array([[        nan,  0.0498314 ,  0.        , ...,  1.30392157,
         0.        ,  0.16764706],
       [ 0.0498314 ,         nan,  0.        , ...,  0.53651805,
         0.03529724,  0.0816879 ],
       [ 0.        ,  0.        ,         nan, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 1.30392157,  0.53651805,  0.        , ...,         nan,
         0.73888889,  1.9       ],
       [ 0.        ,  0.03529724,  0.        , ...,  0.73888889,
                nan,  0.15833333],
       [ 0.16764706,  0.0816879 ,  0.        , ...,  1.9       ,
         0.15833333,         nan]])

In [59]:
cij_tt.shape

(43, 43)

In [61]:
np.isnan(cij_tt).any()

False

In [62]:
tdf = '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site03_Z3000__B1235_cat_P02_S03_1/topology/clique_top/2016-08-23T1548/B1235_cat_P02_S03_1-good-50ms-stim/B1235_cat_P02_S03_1-good-50ms-stim-2016-08-23T1548-CliqueTop--stim-A_scaled_burung-bettiPersistence.pkl'

In [65]:
import pickle
tdf_f = open(tdf, 'r')
tf = pickle.load(tdf_f)

In [66]:
tf

{'0': [[0, [43]]],
 '1': [[0, [44]]],
 '10': [[0, [39]]],
 '11': [[0, [37]]],
 '12': [[0, [38]]],
 '13': [[0, [42]]],
 '14': [[0, [42]]],
 '15': [[0, [42]]],
 '16': [[0, [40]]],
 '17': [[0, [41]]],
 '18': [[0, [41]]],
 '19': [[0, [38]]],
 '2': [[0, [44]]],
 '3': [[0, [44]]],
 '4': [[0, [42]]],
 '5': [[0, [42]]],
 '6': [[0, [39]]],
 '7': [[0, [38]]],
 '8': [[0, [38]]],
 '9': [[0, [43]]]}